In [2]:
import sys
sys.path.append('c:/Users/Sarthak/Documents/GitHub/business_assistant')

from backend.services.rag_embedding_invoice import search_invoices
import os
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
def test_search(query, user_id=None):
    print(f"Query: {query}")
    print(f"User ID: {user_id}")
    print("-" * 50)
    
    results = search_invoices(query, user_id, top_k=3)
    
    for i, result in enumerate(results, 1):
        print(f"Result {i} (Score: {result['score']:.3f}):")
        metadata = result['metadata']
        print(f"  Type: {metadata['type']}")
        print(f"  Vendor: {metadata['vendor_name']}")
        
        if metadata['type'] == 'invoice':
            print(f"  Invoice: {metadata['invoice_number']}")
            print(f"  Total: {metadata['total']} {metadata['currency']}")
            print(f"  Date: {metadata['invoice_date']}")
        else:
            print(f"  Description: {metadata['description']}")
            print(f"  Quantity: {metadata['quantity']}")
            print(f"  Line Total: {metadata['line_total']}")
        print()


In [4]:
test_search("Appario Retail invoices")

Query: Appario Retail invoices
User ID: None
--------------------------------------------------


c:\Users\Sarthak\anaconda3\envs\venv\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [ ]:
# generate_eval_queries.py - Updated
def generate_queries_from_db():
    db = SessionLocal()
    try:
        result = db.execute(text("""
            SELECT 
                i.id as invoice_id,
                i.order_id,
                i.contact_name,
                i.total_price,
                i.city,
                i.country,
                li.product_name,
                li.quantity,
                ROW_NUMBER() OVER (PARTITION BY i.id ORDER BY li.id) - 1 as item_index
            FROM invoices i
            LEFT JOIN line_items li ON i.id = li.invoice_id
            ORDER BY i.id, li.id
        """))
        
        # Generate queries using invoice_id (database ID) for relevant_ids
        queries = []
        
        for row in rows[:10]:  # First 10 rows
            queries.extend([
                {
                    "query": f"Show me the invoice with Order ID {row.order_id}",
                    "relevant_ids": [f"invoice_{row.invoice_id}"]
                },
                {
                    "query": f"Find the line item for {row.product_name}",
                    "relevant_ids": [f"item_{row.invoice_id}_{row.item_index}"]
                }
            ])
        
        return queries


NameError: name '__file__' is not defined